In [19]:
import requests
from requests.auth import HTTPProxyAuth
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
from random import randint
import pandas as pd
import re
import socks
import socket
import sqlite3
from stem import Signal
from stem.control import Controller 
import numpy as np

In [2]:
## Initizlization

#Create SQLite
conn = sqlite3.connect('lyrics.db')

In [ ]:
## Steps for Git
Git add <file> 
Git commit -m "comments" 
Git push origin master

## Scrap Cyber Hymnal

In [3]:
def set_new_ip():
    """Change IP using TOR"""
    with Controller.from_port(port=9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM)


In [4]:
#start tor :  tor & 
#netstat -tupln 
# http://sacharya.com/crawling-anonymously-with-tor-in-python/
# socks.setdefaultproxy(proxy_type=socks.PROXY_TYPE_SOCKS5, addr="127.0.0.1", port=9050)
# socket.socket = socks.socksocket
session = requests.session()
session.proxies = {'https':'socks5://127.0.0.1:9050','http':'socks5://127.0.0.1:9050'}


In [14]:
set_new_ip()
print(session.get("http://icanhazip.com").text)
set_new_ip()
print(session.get("http://icanhazip.com").text)

5.135.158.101

94.23.173.249



In [5]:
# auth = HTTPProxyAuth("username", "password")
user_agent_list = ['Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1623.0 Safari/537.36',
                   'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 7.0; InfoPath.3; .NET CLR 3.1.40767; Trident/6.0; en-IN)',
                   'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0'
                  ]
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [6]:
for i in range(1,10):
  rand_user_agent = randint(0,len(user_agent_list)-1)
  headers['User-Agent'] = user_agent_list[rand_user_agent]
  print(headers['User-Agent'])

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36
Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 7.0; InfoPath.3; .NET CLR 3.1.40767; Trident/6.0; en-IN)
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36
Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36
Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0
Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36


In [33]:
columns = ['url','title','status']
df = pd.DataFrame(columns=columns)

url = 'http://cyberhymnal.org/ttl/ttl-'
for i in range(ord('x'), ord('z')+1):
    #    print(url+chr(i)+'.htm')
    try:
        hurl = url+chr(i)+'.htm'
        pause = randint(3,8)
        time.sleep(pause)
        response = session.get(hurl, headers=headers)
        data = response.text
        soup = BeautifulSoup(data,"lxml")
        div =  soup.find('div',attrs={"class":"single-entry"})
        links = div.findAll("a")
        #Loop through the link in each page
        for i in links:
            url_link =  i.get("href").replace("..","http://cyberhymnal.org")
            song_name = i.get_text()
            status = 0
            # row = [url_link,song_name,status]
            print(row)
            df.loc[len(df)] = row  
    except pageError as e:
        print(hurl)
        print(e)

len(df)
# df.to_sql('lyrics_url',conn,if_exists='replace')

NameError: name 'pageError' is not defined

In [28]:
df['first_alphabet'] = df['title'].str[0]

In [29]:
df

,url,title,status,first_alphabet
0,http://cyberhymnal.org/htm/a/b/abbafath.htm,"ABBA, FATHER! WE APPROACH THEE",0.0,A
1,http://cyberhymnal.org/htm/a/b/abidinme.htm,"ABIDE IN ME, O LORD",0.0,A
2,http://cyberhymnal.org/htm/a/i/abinthee.htm,ABIDE IN THEE,0.0,A
3,http://cyberhymnal.org/htm/a/b/abidenot.htm,ABIDE NOT IN THE REALM OF DREAMS,0.0,A
4,http://cyberhymnal.org/htm/a/o/aodjesus.htm,"ABIDE, O DEAREST JESUS",0.0,A
5,http://cyberhymnal.org/htm/a/b/abidewme.htm,ABIDE WITH ME,0.0,A
6,http://cyberhymnal.org/htm/a/b/abidewmt.htm,"ABIDE WITH ME, ’TIS EVENTIDE",0.0,A
7,http://cyberhymnal.org/htm/a/w/awuosavr.htm,"ABIDE WITH US, OUR SAVIOR",0.0,A
8,http://cyberhymnal.org/htm/a/b/abidewus.htm,"ABIDE WITH US, THE DAY IS WANING",0.0,A
9,http://cyberhymnal.org/htm/a/b/able2del.htm,ABLE TO DELIVER,0.0,A


In [32]:
pd.pivot_table(df,index=['first_alphabet'],values=['status'],aggfunc='count')

,status
first_alphabet,
A,325
B,311
C,367
D,133
E,88
F,236
G,334
H,584
I,620


In [ ]:

cur = conn.cursor()
cur.execute('delete from test')
conn.commit

pd.read_sql_query("select * from test;",conn)

In [32]:
# Song Pages
# url = 'http://cyberhymnal.org/htm/b/e/bestrong.htm'
url = 'http://cyberhymnal.org/htm/a/b/abidinme.htm'
response = requests.get(url, headers=headers)
data = response.text
soup = BeautifulSoup(data,"lxml")

In [33]:
soup

<?xml version="1.0" encoding="ISO-8859-2" ?><!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd">
<html xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<!-- Copyright 1999-2007 The Cyber Hymnal -->
<head>
<title>Abide in Me, O Lord</title>
<meta content="Words: Harriet Stowe, 1855. Music: Templi Carmina, 1853." name="description"/>
<meta content="Harriet Stowe, Templi Carmina" name="keywords"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<script src="../../../js/base.js"></script>
<script src="../../../js/hymn.js"></script>
<script>init(3,"c/u/b/cuba","abbafath","../i/abinthee")</script>
</head>
<body>
<script>start("ABIDE IN ME, O LORD","He that keepeth His commandments dwelleth in Him, and He in him. And hereby we know that He abideth in us, by the Spirit which He hath given us.","1 John 3:24")</script>
<table class="intro"><tbody><tr>
<script>portrait("Harriet B. Stowe (1812-1896)","s/t/stowe_hb",198,293)</script

In [34]:
song_composers = soup.body.table.find("td").findAll("p")

sc = []
for s in song_composers:
    sc.append(s.get_text().replace("&shy;",""))
    print(s.get_text().replace("&shy;",""))
# song_music =  soup.find('span',attrs={"class":"music"})
# print(song_music)

song_lyrics =  soup.find('div',attrs={"class":"lyrics"})
print(song_lyrics)

Words: bio("Harriet B. Stowe","s/t/o/stowe_hb"), in the Ply­mouth Col­lect­ion, 1855.
Music: Cuba, in Temp­li Car­mi­na, by bio("George Kingsley","k/i/n/kingsley_g") (North­amp­ton, Mass­a­chu­setts: 1853)lmn("c/u/Cuba").
<div class="lyrics">
<p>Abide in me, O Lord, and I in Thee,<br/>
From this good hour, oh, leave me nevermore;<br/>
Then shall the discord cease, the wound be healed,<br/>
The lifelong bleeding of the soul be o’er.</p>
<p>Abide in me; o’ershadow by Thy love<br/>
Each half formed purpose and dark thought of sin;<br/>
Quench ere it rise each selfish, low desire,<br/>
And keep my soul as Thine, calm and divine.</p>
<p>As some rare perfume in a vase of clay,<br/>
Pervades it with a fragrance not its own,<br/>
So, when Thou dwellest in a mortal soul,<br/>
All Heaven’s own sweetness seems around it thrown.</p>
<p>Abide in me; there have been moments blest<br/>
When I have heard Thy voice and felt Thy power;<br/>
Then evil lost its grasp; and passion, hushed,<br/>
Owned the d

In [37]:
print(sc[0])
composer = re.findall("bio\(\"(.*)\",",sc[0])
year = re.findall("\d{4}",sc[0])
print("Composer is ",composer[0]," Year ",year[0])
final = re.sub("bio\(\"(.*)\",.*?\)",r"\1",sc[0])
print(final)

print(sc[1])
music = re.sub("bio\(\"(.*)\",.*?\)",r"\1",sc[1])
music = re.sub("lmn\(\".*","",music)
print(music)

Words: bio("Harriet B. Stowe","s/t/o/stowe_hb"), in the Ply­mouth Col­lect­ion, 1855.
Composer is  Harriet B. Stowe  Year  1855
Words: Harriet B. Stowe, in the Ply­mouth Col­lect­ion, 1855.
Music: Cuba, in Temp­li Car­mi­na, by bio("George Kingsley","k/i/n/kingsley_g") (North­amp­ton, Mass­a­chu­setts: 1853)lmn("c/u/Cuba").
Music: Cuba, in Temp­li Car­mi­na, by George Kingsley (North­amp­ton, Mass­a­chu­setts: 1853)


## Reference Scraping 

In [ ]:
url = 'http://www.whoishostingthis.com/tools/user-agent/'
response = requests.get(url, headers=headers,auth=auth)
data = response.text
soup = BeautifulSoup(data,"lxml")
useragent = soup.find(attrs={"class":"info-box user-agent"})
ip = soup.find(attrs={"class":"info-box ip"})
print(useragent.text)
print(headers["User-Agent"])
print(ip.span.text)

In [ ]:
### Example Code for using Sessions
# s = requests.Session()
# s.get(url,headers=headers)
# print(s.cookies)
# r = s.post(url, data=payload,auth=auth,headers=headers,cookies=cookie)

In [ ]:
cookie = {'session_name':'session','cookie_1': 'cookie_session_1'}
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
           'header_2':'header_value_2'
          }


payload = {'input1':'value1',
           'input2': 'value2',
          }

r = requests.post(url, data=payload,auth=auth,headers=headers,cookies=cookie)

data = r.text
soup = BeautifulSoup(data,"lxml")
#Debug
r.request.headers
r.headers

file = open("output.html","w") 
file.write(soup.prettify()) 
file.close() 

In [ ]:
#Create SQLite
conn = sqlite3.connect('lyrics.db')
eumo = pd.read_csv('EU_MO.csv')
eumo.to_sql('test',conn,if_exists='replace')

cur = conn.cursor()
cur.execute('delete from test')
conn.commit

pd.read_sql_query("select * from test;",conn)

## Scrap AZ Lyrics

## Scrap Company Details

In [141]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
from random import randint
#Websites
#https://www.sgpbusiness.com

#Read up <site>/robots.txt

In [142]:
search_url = "http://www.yelu.sg/category/postal-services/"
base_url = "http://www.yelu.sg"

html = urlopen(search_url)
bsObj = BeautifulSoup(html.read(),"lxml")

In [143]:
page = bsObj.findAll("a",{"class":"page_no"})

max = 0 
for i in page:
    page_no = int(i.get_text())
    if (page_no > max):
        max = page_no
print(max)
# page.findAll("div")

9


In [ ]:
f1 = open('scrap.txt','w')
for x in range(1,max+1):
    pg_search = search_url+str(x)
    print(search_url+str(x))
    #Loop through the pages
    pg_url = urlopen(pg_search)
    pgObj = BeautifulSoup(pg_url.read(),"lxml")
    time.sleep(5)
    pages = pgObj.findAll("h4")
    #Loop through the link in each page
    for i in pages:
        ref = i.a.get("href")
        ran_sleep = randint(10,12)
        time.sleep(ran_sleep)
        coy = urlopen(base_url+ref)
        coyObj = BeautifulSoup(coy.read(),"lxml")
        for s in scrapitem:
          try:
            if len(s.find_all('div')) == 0 :
              header = s.find('span').get_text()
              description = s.get_text()[len(header):]
            else: 
              header = s.find_all('div')[0].get_text()  
              if s.find('span') == None :
                description = s.find_all('div')[1].get_text()
              else : 
                description = s.find('span').get_text()    
            #f1.write(base_url+ref)
            f1.write(header,":",description,"\n")
            if header == 'E-mail' :
                break
          except AttributeError as e:
            continue
          except Exception as ea:
            continue
f1.close()

http://www.yelu.sg/category/postal-services/1
http://www.yelu.sg/category/postal-services/2
http://www.yelu.sg/category/postal-services/3
http://www.yelu.sg/category/postal-services/4
http://www.yelu.sg/category/postal-services/5

In [88]:
# t.find_all('div')[0].get_text()
# t.find_all('div')[1].get_text()
x = t.find('span')
if (x == None):
  print("No Span")
else :
  print("Yes")
# t.div.find_next_siblings()

No Span


In [98]:
pg_url = urlopen("http://www.yelu.sg/category/postal-services/9")

In [101]:
pgObj = BeautifulSoup(pg_url.read(),"lxml")
pages = pgObj.findAll("h4")
pages

[]

In [6]:
def getTitle(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        return None
    try:
        bsObj = BeautifulSoup(html.read())
        title = bsObj.body.h1
    except AttributeError as e:
        return None
    return title

In [63]:
time.sleep(3)